# fitness-eval-app — SQLite Database Explorer

This notebook connects to the SQLite database created in **Phase 2** and displays
all four tables for inspection and reference.

**Tables:**
| Table | Description |
|-------|-------------|
| `coaches` | Registered coach accounts (bcrypt passwords) |
| `clients` | Clients owned by a coach — strict per-coach isolation |
| `body_measurements` | Timestamped measurement snapshots per client |
| `assessments` | Timestamped assessment result blobs per client |

> **Run order:** Execute cells top-to-bottom. The setup cell creates the DB
> and migrates data from `clients.json` / `coaches.json` if not already done.

## 1 — Setup

In [ ]:
import asyncio
import json
import sqlite3
import sys
from pathlib import Path

import pandas as pd

# ── Resolve paths ─────────────────────────────────────────────────────────────
REPO_ROOT  = Path("../").resolve()
BACKEND    = REPO_ROOT / "backend"
DB_PATH    = BACKEND / "data" / "fitness.db"

# Add backend to sys.path so we can import app modules.
if str(BACKEND) not in sys.path:
    sys.path.insert(0, str(BACKEND))

print(f"Repo root : {REPO_ROOT}")
print(f"Backend   : {BACKEND}")
print(f"DB path   : {DB_PATH}")
print(f"DB exists : {DB_PATH.exists()}")

## 2 — Create Tables + Run Migration

Safe to run repeatedly — migration skips if the `coaches` table is already populated.

In [ ]:
# Import ORM models to register them with Base.metadata.
import app.db_models  # noqa: F401  (side-effect: registers tables)
from app.database import AsyncSessionLocal, create_tables
from app.migrate_json_to_db import run_migration_if_needed

async def setup_db() -> None:
    """Create tables and seed data from JSON files (idempotent)."""
    await create_tables()
    async with AsyncSessionLocal() as db:
        await run_migration_if_needed(db)

await setup_db()
print(f"DB ready: {DB_PATH}  ({DB_PATH.stat().st_size / 1024:.1f} KB)")

## 3 — Helper: read any table into a DataFrame

In [ ]:
def read_table(table: str, limit: int | None = None) -> pd.DataFrame:
    """Read a SQLite table into a pandas DataFrame.

    Args:
        table: Table name.
        limit: Optional row cap (None = all rows).

    Returns:
        DataFrame with all columns.
    """
    sql = f"SELECT * FROM {table}"  # noqa: S608 — notebook only, no user input
    if limit:
        sql += f" LIMIT {limit}"
    with sqlite3.connect(DB_PATH) as con:
        return pd.read_sql_query(sql, con)


def table_info(table: str) -> pd.DataFrame:
    """Return PRAGMA table_info for column definitions."""
    with sqlite3.connect(DB_PATH) as con:
        return pd.read_sql_query(f"PRAGMA table_info({table})", con)


print("Helper functions ready.")

## 4 — `coaches` table

In [ ]:
coaches = read_table("coaches")

# Don't expose the hash — mask it for display.
coaches_display = coaches.copy()
coaches_display["hashed_password"] = "[bcrypt hash]" 

print(f"Rows: {len(coaches_display)}")
coaches_display

In [ ]:
# Column definitions
table_info("coaches")

## 5 — `clients` table

In [ ]:
clients = read_table("clients")

# Parse goals JSON for readability.
clients_display = clients.copy()
clients_display["goals"] = clients_display["goals"].apply(
    lambda g: ", ".join(json.loads(g)) if g else ""
)

print(f"Rows: {len(clients_display)}")
clients_display

In [ ]:
# Column definitions
table_info("clients")

## 6 — `body_measurements` table

In [ ]:
measurements = read_table("body_measurements")

print(f"Rows: {len(measurements)}")
measurements

In [ ]:
# Column definitions
table_info("body_measurements")

## 7 — `assessments` table

In [ ]:
assessments = read_table("assessments")

# Parse results_json to show metric count and rating summary instead of raw blob.
def _summarise_results(blob: str) -> str:
    try:
        results = json.loads(blob)
        parts = [f"{r['test_name']}: {r['rating']}" for r in results]
        return " | ".join(parts)
    except Exception:
        return blob

assessments_display = assessments.copy()
assessments_display["results_summary"] = assessments_display["results_json"].apply(_summarise_results)
assessments_display = assessments_display.drop(columns=["results_json"])

print(f"Rows: {len(assessments_display)}")
assessments_display

In [ ]:
# Column definitions
table_info("assessments")

## 8 — Cross-table join: clients with their coach name

In [ ]:
sql = """
SELECT
    cl.id            AS client_id,
    co.username      AS coach,
    cl.name          AS client_name,
    cl.age,
    cl.gender,
    cl.height_cm,
    cl.saved_at,
    COUNT(DISTINCT a.id)  AS assessment_count,
    COUNT(DISTINCT bm.id) AS measurement_count
FROM clients cl
JOIN coaches co ON co.id = cl.coach_id
LEFT JOIN assessments a   ON a.client_id  = cl.id
LEFT JOIN body_measurements bm ON bm.client_id = cl.id
GROUP BY cl.id
ORDER BY co.username, cl.saved_at DESC
"""

with sqlite3.connect(DB_PATH) as con:
    joined = pd.read_sql_query(sql, con)

print(f"Rows: {len(joined)}")
joined